In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [ ]:
from IPython.core.async_helpers import sys
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

In [ ]:
raw_datasets

In [ ]:
raw_datasets = raw_datasets.shuffle(seed=42)
raw_datasets["train"] = raw_datasets["train"].select(range(20000))
raw_datasets["validation"] = raw_datasets["validation"].select(range(500))

preprocessing

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("Hello, this is a sentence!")

In [ ]:
max_input_length = 64
max_target_length = 64
source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples["translation"]]
  targets = [ex[target_lang] for ex in examples["translation"]]
  model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
  )
  labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True
    )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])

In [ ]:
print(tokenizer)
print(type(tokenizer))


In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
#num_train_epochs = 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, return_tensors="tf")

In [ ]:
#gneration_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=64)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size = batch_size,
    shuffle=True,
    collate_fn = data_collator,
)

In [ ]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

In [ ]:
model.save_pretrained("translator_model")
tokenizer.save_pretrained("translator_model")


testing

In [ ]:
pip install sacremoses

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("translator_model")
model = TFAutoModelForSeq2SeqLM.from_pretrained("translator_model")

In [ ]:
input_text = "Goodmorning"
tokenized = tokenizer([input_text], return_tensors = 'np')
out = model.generate(**tokenized, max_length = 64)
print(out)

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))